We need to break out columns into the pre-stop observables - e.g. reason for stop, age / weight / build, precinct, xcoord/ycoord
from the during-stop features

In [321]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
import data_cleaner as dc
import clean_cat_values
import data_modeler as dm
import importlib
import data_dicts
importlib.reload(data_dicts)
importlib.reload(clean_cat_values)
importlib.reload(dc)
importlib.reload(dm)

<module 'data_modeler' from '/Users/brad/Documents/Scripts/flatiron/stop-and-frisk/data_modeler.py'>

In [322]:
df = dc.load_full_sqf(force=True)

Loading 2003...
Loading 2004...
Loading 2005...
Loading 2006...
Loading 2007...
Loading 2008...
Loading 2009...
Loading 2010...
Loading 2011...
Loading 2012...
Loading 2013...
Loading 2014...
Loading 2015...
Loading 2016...
Loading 2017...
Loading 2018...
Done.


In [93]:
pd.options.display.max_rows=114
pd.options.display.max_colwidth=100
pd.options.display.max_columns=114

In [326]:
full_df = df.copy()
df = df.sample(5000)

In [147]:
df.describe()

,year,pct,ser_num,perobs,perstop,explnstp,othpers,arstmade,sumissue,compyear,comppct,offunif,frisked,searched,contrabn,adtlrept,pistol,riflshot,asltweap,knifcuti,machgun,othrweap,pf_hands,pf_wall,pf_grnd,pf_drwep,pf_ptwep,pf_baton,pf_hcuff,pf_pepsp,pf_other,radio,ac_rept,ac_inves,rf_vcrim,rf_othsw,ac_proxm,rf_attir,cs_objcs,cs_descr,cs_casng,cs_lkout,rf_vcact,cs_cloth,cs_drgtr,ac_evasv,ac_assoc,cs_furtv,rf_rfcmp,ac_cgdir,rf_verbl,cs_vcrim,cs_bulge,cs_other,ac_incid,ac_time,rf_knowl,ac_stsnd,ac_other,sb_hdobj,sb_outln,sb_admis,sb_other,rf_furt,rf_bulg,age,ht_feet,ht_inch,weight,premtype,state,addrpct,beat,xcoord,ycoord,linecm,detailcm,height,wepfound,detail1_
count,5000.000000,5000.000000,5000.000000,4992.000000,4999.000000,5000.000000,5000.000000,5000.000000,5000.000000,4976.0,4976.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,4997.000000,5000.000000,5000.000000,4999.000000,0.0,0.0,4230.000000,1461.000000,4.066000e+03,4066.000000,3706.000000,4231.000000,5000.000000,5000.000000,525.000000
mean,2008.555200,68.199200,6096.495800,2.580655,5.534107,0.997000,0.226400,0.062400,0.062800,0.0,0.0,0.724200,0.522200,0.084600,0.019000,0.003200,0.001600,0.0,0.000200,0.007400,0.0,0.001600,0.174200,0.030800,0.006000,0.002200,0.003200,0.000200,0.036000,0.000400,0.011400,0.231800,0.128000,0.12260,0.105600,0.076200,0.210600,0.075600,0.026200,0.188800,0.274000,0.158600,0.075400,0.03920,0.096800,0.166800,0.040000,0.437800,0.08300,0.231000,0.005000,0.07980,0.085400,0.205000,0.544200,0.370200,0.014000,0.021400,0.03920,0.035400,0.006000,0.003200,0.040800,0.328400,0.081400,28.026016,5.176800,6.364200,167.572515,NaN,NaN,67.786998,8.202601,1.004483e+06,204417.800295,0.966811,38.209643,68.485800,0.000400,36.441905
std,2.829303,32.675456,6158.218398,4.246915,5.703946,0.054696,0.418543,0.241905,0.242627,0.0,0.0,0.446961,0.499557,0.278313,0.136538,0.056484,0.039972,0.0,0.014142,0.085713,0.0,0.039972,0.379319,0.172793,0.077235,0.046857,0.056484,0.014142,0.186309,0.019998,0.106171,0.422024,0.334123,0.32801,0.307356,0.265344,0.407776,0.264383,0.159746,0.391389,0.446054,0.365339,0.264062,0.19409,0.295715,0.372834,0.195979,0.496166,0.27591,0.421515,0.070541,0.27101,0.279504,0.403742,0.498092,0.482906,0.117502,0.144728,0.19409,0.184807,0.077235,0.056484,0.197846,0.469678,0.273476,14.797760,0.432642,3.376187,37.741267,NaN,NaN,32.562424,6.499422,3.708253e+04,28974.670248,0.179155,25.572005,3.978865,0.019998,25.112399
min,2003.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,1.000000,1.000000,5.907840e+05,124340.000000,0.000000,6.000000,0.000000,0.000000,9.000000
25%,2006.000000,42.000000,1824.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

# Modeling on the yes-no columns

In [182]:
y = df.arstmade

In [183]:
X_y_n = df.select_dtypes(include=['int8']).drop(columns='arstmade')

In [184]:
split = dm.load_split(X_y_n, y)

## Balance classes

In [185]:
split['y_train'].value_counts(normalize=True)

0    0.934933
1    0.065067
Name: arstmade, dtype: float64

In [186]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(split['X_train'], split['y_train']) 
y_train_resampled.value_counts()

1    3506
0    3506
Name: arstmade, dtype: int64

In [187]:
logit = LogisticRegression(solver='saga', penalty='l1', max_iter=4000)

In [188]:
logit.fit(X_train_resampled, y_train_resampled)
y_hat_train = logit.predict(split['X_train'])
residuals = np.abs(split['y_train'] - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    3370
1     380
Name: arstmade, dtype: int64
0    0.898667
1    0.101333
Name: arstmade, dtype: float64


In [189]:
pd.Series(y_hat_train).value_counts()

0    3248
1     502
dtype: int64

In [190]:
y_hat_test = logit.predict(split['X_test'])
residuals = np.abs(split['y_test'] - y_hat_test)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    1119
1     131
Name: arstmade, dtype: int64
0    0.8952
1    0.1048
Name: arstmade, dtype: float64


In [191]:
logit.coef_

array([[ 1.15422117, -0.63778506, -2.76921075, -0.43906889,  1.53664947,
         4.71244694,  5.92921729, -2.63908453,  0.        ,  0.        ,
         0.        , -0.02803667,  0.        ,  0.        , -1.20198606,
        -0.6356687 , -2.01897471, -1.57564569, -0.33389638,  0.21683498,
         2.36879078,  0.        , -2.45493293, -1.06587496, -0.05039847,
        -2.07659287,  0.33961796, -1.04238158, -0.96452628, -0.92649511,
        -3.13833436, -0.80436542, -2.71756743, -2.22544411, -0.58363806,
        -0.37144361, -1.7388306 ,  0.        , -2.33176135, -2.07317647,
        -0.25440522, -0.63755845,  0.        , -2.14312206, -2.25814912,
        -1.29766975, -0.73734479, -0.16672737, -1.79909294, -0.64480502,
        -2.15552186, -4.068843  , -1.81998161,  2.25932235,  0.53537669,
        -0.57391598, -2.2129033 ,  0.        ]])

In [192]:
split['X_train'].columns

Index(['explnstp', 'othpers', 'sumissue', 'offunif', 'frisked', 'searched',
       'contrabn', 'adtlrept', 'pistol', 'riflshot', 'asltweap', 'knifcuti',
       'machgun', 'othrweap', 'pf_hands', 'pf_wall', 'pf_grnd', 'pf_drwep',
       'pf_ptwep', 'pf_baton', 'pf_hcuff', 'pf_pepsp', 'pf_other', 'radio',
       'ac_rept', 'ac_inves', 'rf_vcrim', 'rf_othsw', 'ac_proxm', 'rf_attir',
       'cs_objcs', 'cs_descr', 'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth',
       'cs_drgtr', 'ac_evasv', 'ac_assoc', 'cs_furtv', 'rf_rfcmp', 'ac_cgdir',
       'rf_verbl', 'cs_vcrim', 'cs_bulge', 'cs_other', 'ac_incid', 'ac_time',
       'rf_knowl', 'ac_stsnd', 'ac_other', 'sb_hdobj', 'sb_outln', 'sb_admis',
       'sb_other', 'rf_furt', 'rf_bulg', 'wepfound'],
      dtype='object')

In [193]:
coef = pd.DataFrame(logit.coef_)
coef.columns = split['X_train'].columns
coef.T

,0
explnstp,1.154221
othpers,-0.637785
sumissue,-2.769211
offunif,-0.439069
frisked,1.536649
searched,4.712447
contrabn,5.929217
adtlrept,-2.639085
pistol,0.000000
riflshot,0.000000


In [60]:
df.groupby(by=['sumissue','arstmade']).count().iloc[:,0]

sumissue  arstmade
0         0           4372
          1            284
1         0            332
          1             12
Name: year, dtype: int64

## Only pre-stop observables

In [205]:
X_pre_stop = df[list(set(dc.Y_N_COLS).intersection(dm.PRE_STOP_OBSERVABLES))]

In [221]:
split, logit = dm.split_and_logit(X_y_n, y)

Train
 0    3370
1     380
Name: arstmade, dtype: int64
0    0.898667
1    0.101333
Name: arstmade, dtype: float64
Test
 0    1117
1     133
Name: arstmade, dtype: int64
0    0.8936
1    0.1064
Name: arstmade, dtype: float64


In [206]:
split, logit = dm.split_and_logit(X_pre_stop, y)

Train
 0    2503
1    1247
Name: arstmade, dtype: int64
0    0.667467
1    0.332533
Name: arstmade, dtype: float64
Test
 0    824
1    426
Name: arstmade, dtype: int64
0    0.6592
1    0.3408
Name: arstmade, dtype: float64


In [219]:
balanced_accuracy_score(split['y_test'], np.zeros(split['y_test'].shape))

0.5

In [222]:
balanced_accuracy_score(split['y_test'], logit.predict(split['X_test']))

0.7847765715728352

In [225]:
df.build.value_counts()

M    2947
T    1530
H     430
Z      74
U      19
Name: build, dtype: int64

In [227]:
df.race.value_counts(normalize=True)

B                                 0.5180
Q                                 0.2442
W                                 0.0906
P                                 0.0650
Z                                 0.0370
A                                 0.0324
U                                 0.0044
I                                 0.0036
BLACK                             0.0028
WHITE HISPANIC                    0.0012
WHITE                             0.0004
BLACK HISPANIC                    0.0004
X                                 0.0000
ASIAN/PAC.ISL                     0.0000
ASIAN / PACIFIC ISLANDER          0.0000
AMERICAN INDIAN/ALASKAN NATIVE    0.0000
AMER IND                          0.0000
Name: race, dtype: float64

In [228]:
full_df.race.value_counts(normalize=True)

B                                 0.515874
Q                                 0.242384
W                                 0.098652
P                                 0.061078
Z                                 0.038161
A                                 0.030701
U                                 0.004500
I                                 0.003928
BLACK                             0.002473
WHITE HISPANIC                    0.000960
X                                 0.000420
WHITE                             0.000402
BLACK HISPANIC                    0.000379
ASIAN / PACIFIC ISLANDER          0.000043
ASIAN/PAC.ISL                     0.000040
AMERICAN INDIAN/ALASKAN NATIVE    0.000003
AMER IND                          0.000002
Name: race, dtype: float64

In [229]:
full_df[full_df.year==2017].race.value_counts(normalize=True)

BLACK                             0.579202
WHITE HISPANIC                    0.226084
WHITE                             0.088623
BLACK HISPANIC                    0.086702
ASIAN/PAC.ISL                     0.018566
AMER IND                          0.000823
Z                                 0.000000
AMERICAN INDIAN/ALASKAN NATIVE    0.000000
ASIAN / PACIFIC ISLANDER          0.000000
B                                 0.000000
I                                 0.000000
X                                 0.000000
P                                 0.000000
Q                                 0.000000
U                                 0.000000
W                                 0.000000
A                                 0.000000
Name: race, dtype: float64

In [231]:
df17 = dc.load_sqf(2017, convert=False)

Loading 2017...


In [236]:
df17.SUSPECT_EYE_COLOR.value_counts(normalize=True)

BRO    0.850370
BLK    0.089673
ZZZ    0.022484
BLU    0.015599
HAZ    0.009237
GRN    0.008976
GRY    0.001394
BR     0.001133
OTH    0.000349
MED    0.000261
THN    0.000174
HEA    0.000174
MUL    0.000087
MAR    0.000087
Name: SUSPECT_EYE_COLOR, dtype: float64

In [237]:
full_df.eyecolor.value_counts(normalize=True)

BR     8.834214e-01
BK     6.924873e-02
BL     1.825134e-02
XX     7.201152e-03
GR     6.869421e-03
HA     6.587046e-03
Z      6.485776e-03
GY     1.162339e-03
DF     4.294617e-04
ZZ     1.592784e-04
MA     1.402043e-04
VI     2.123712e-05
MC     1.238832e-05
P      6.095839e-06
PK     2.556320e-06
MUL    1.179840e-06
UN     1.966400e-07
MAR    1.966400e-07
Name: eyecolor, dtype: float64

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 3079204 to 1353209
Columns: 118 entries, year to detail1_
dtypes: Int64(7), category(20), datetime64[ns](1), float64(14), int8(59), object(17)
memory usage: 1.9+ MB


In [239]:
df.build.value_counts()

M    2947
T    1530
H     430
Z      74
U      19
Name: build, dtype: int64

In [240]:
ONE_HOT_ENCODE = ['build', 'pct']

In [257]:
df.select_dtypes('category').columns.to_list()

['recstat',
 'inout',
 'trhsloc',
 'typeofid',
 'officrid',
 'offverb',
 'offshld',
 'sex',
 'race',
 'haircolr',
 'eyecolor',
 'build',
 'addrtyp',
 'rescode',
 'city',
 'sector',
 'dettypcm',
 'forceuse',
 'month',
 'day']

In [325]:
df.month.value_counts().sum()

22205

In [313]:
df.forceuse.value_counts()

DS    48
OT    26
SF    21
SW     9
OR     1
Name: forceuse, dtype: int64

In [314]:
df[~df.forceuse.isna()].year.value_counts()

2012    66
2013    28
2014     6
2015     4
2016     1
2018     0
2017     0
2011     0
2010     0
2009     0
2008     0
2007     0
2006     0
2005     0
2004     0
2003     0
Name: year, dtype: int64

In [315]:
full_df.forceuse.value_counts()

DS    49859
SF    25284
OT    24945
SW     8585
OR     4048
DO     1744
Name: forceuse, dtype: int64

In [291]:
df = dc.clean_categories(df)